<a href="https://colab.research.google.com/github/WalrusEggMan/intro-to-is/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [0]:
import pandas as pd

url = 'https://raw.githubusercontent.com/WalrusEggMan/intro-to-is/master/data.csv'

data = pd.read_csv(url, delimiter=',')
data.head()
data.info()
data.describe()

# New Section

In [0]:
def fix_seniority_in_company(row):
  if row['seniority in role(years)'] > row['seniority (years) in comapny']:
    return int(row['seniority (years) in comapny'])
  else:
    return int(row['seniority in role(years)'])

def fix_seniority_in_role(row):
  if row['seniority in role(years)'] < row['seniority (years) in comapny']:
    return int(row['seniority (years) in comapny'])
  else:
    return int(row['seniority in role(years)'])

data['fix_sen_in_role'] = data.apply(fix_seniority_in_role,axis=1)
data['fix_sen_in_comp'] = data.apply(fix_seniority_in_company,axis=1)
data.head(3)

In [0]:


#data[data.age - data['fixed_sen_in_role'] <= 16]
#Create new df with clean data
#Drop Invalid columns
data.drop('seniority (years) in comapny', axis=1, inplace=True)
data.drop('seniority in role(years)', axis=1, inplace=True)

#Delete rows with wrong seniority in role
clean_data = data[data.age - data['fix_sen_in_role'] > 16]
clean_data = clean_data.reset_index(drop=True)
clean_data.describe()
#All values are not-null values


In [0]:
from pandas.plotting import scatter_matrix
# scatter plot matrix
#rename columns
clean_data.rename(columns={'monthly return on loan': 'm_ret_loan', 
                           'how many children': 'childNo',
                           'grade in last year review (0-10)': 'grade_l_year',
                           'averaged grade of the BSC': 'avg_grade'}, inplace=True)
#scatter_matrix(clean_data,figsize=(10,10))

In [0]:
#Scatter for Age/Salary and Age/seniority in role pairs
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
cmap = sns.cubehelix_palette(dark=.3, light=.8, as_cmap=True)
ax = sns.scatterplot(x="age", y="salary", palette=cmap,
                     data=clean_data[['age', 'salary']])
ax.set_title("Age/Salary")



In [0]:
ax = sns.scatterplot(x="age", y="fix_sen_in_role", palette=cmap,
                     data=clean_data[['age', 'fix_sen_in_role']])
ax.set_title("Age/Seniority in role(years)")

In [0]:
normalized_df=(clean_data-clean_data.min())/(clean_data.max()-clean_data.min())
ax = sns.scatterplot(x="age", y="salary", palette=cmap,
                     data=normalized_df[['age', 'salary']])
ax.set_title("Age/Salary (normalized by minmax)")
#normalized_df.head()


In [0]:
ax = sns.scatterplot(x="age", y="fix_sen_in_role", palette=cmap,
                     data=normalized_df[['age', 'fix_sen_in_role']])
ax.set_title("Age/Seniority in role(normalized by minmax)")

In [0]:
def linear_regression_plot(x_name, y_name):
  # Seperating the data into dependent and independent variables

  # sns.regplot(x="age", y="salary",
  #                      data=clean_data[['age', 'salary']]);

  y = normalized_df[x_name].values
  X = normalized_df[[y_name]].values


  #Splitting the dataset into the Training set and Test set
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

  # LinearRegression
  from sklearn.linear_model import LinearRegression
  # Create linear regression object
  classifier = LinearRegression()
  classifier.fit(X_train, y_train)

  y_pred = classifier.predict(X_test)

  from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
  # The coefficients
  print('Coefficients: \n', classifier.coef_)
  # The mean squared error
  print('Mean squared error: %.2f'
        % mean_squared_error(y_test, y_pred))
  # The mean error
  print('Mean error: %.2f'
        % mean_squared_error(y_test, y_pred, squared=False))
  # The mean absolute error
  print('Mean absolute error: %.2f'
        % mean_absolute_error(y_test, y_pred))
  # The coefficient of determination: 1 is perfect prediction
  print('Coefficient of determination: %.2f'
        % r2_score(y_test, y_pred))

  # Plot outputs
  # Test vs predicted data
  plt.scatter(X_test, y_test,  color='blue')
  plt.plot(X_test, y_pred, color='green', linewidth=3)

  plt.xticks(())
  plt.yticks(())

  plt.show()

linear_regression_plot('age', 'salary')

In [0]:
linear_regression_plot('age', 'fix_sen_in_role')